In [3]:


--**********************************************************************************--
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
-------------------------  DML - Tables  ---------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------


/*********************************************************/
/*********************************************************/
		--  DML - DIM Airports
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM dim.airports) = 0
	BEGIN
INSERT INTO dim.airports (AIRPORT_ID,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE)

SELECT DISTINCT IATA_CODE
      ,AIRPORT
      ,CITY
      ,STATE
      ,COUNTRY
      ,LATITUDE
      ,LONGITUDE
  FROM Flights_Reporting.dbo.airports

END

/*********************************************************/
/*********************************************************/
		--  DML for DIM Airlines
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM dim.airlines) = 0
BEGIN
	INSERT INTO dim.airlines (AIRLINE_ID, AIRLINE)
		SELECT DISTINCT IATA_CODE
			, AIRLINE 
		FROM Flights_Reporting.dbo.airlines
END

/*********************************************************/
/*********************************************************/
		--  DML for DIM CANCELLATIONS
/*********************************************************/
/*********************************************************/
IF (SELECT COUNT(*) FROM dim.airlines) = 0
BEGIN
INSERT INTO dim.cancellations(CANCELID, Reasoning)

		SELECT DISTINCT CANCELLATION_REASON 'CancelID'
		, CASE	WHEN CANCELLATION_REASON = 'A' THEN 'Airline/Carrier'
				WHEN CANCELLATION_REASON = 'B' THEN 'Weather'
				WHEN CANCELLATION_REASON = 'C' THEN 'National Air System'
				WHEN CANCELLATION_REASON = 'D' THEN 'Security'
			ELSE NULL
			END 'Description'
		FROM Flights_Reporting.dbo.flights
		WHERE CANCELLATION_REASON in ('A', 'B', 'C', 'D')
		ORDER BY CancelID
END

/*********************************************************/
/*********************************************************/
		--  DML for DIM Calendar
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM dim.Calendar) = 0
	BEGIN
-- Variable Declaration
DECLARE @StartDate DATE = '2015-01-01'
DECLARE @EndDate DATE = '2015-12-31'
DECLARE @date DATE = @StartDate
DECLARE @DayID INT = (DATEPART(YEAR, @StartDate)-1900)*1000 + DATEPART(DY, @StartDate)


--Populate Calendar
WHILE @Date <= @EndDate
	BEGIN
		INSERT INTO dim.Calendar (pkCalendar, DateValue, fYear, fMonthNu, fMonth, fDay, fDayName, IsWeekDay, WeekDay)
			VALUES (
				@DayID
				, @date 
				, YEAR(@date)
				, MONTH(@date)
				, DATENAME(MONTH, @date)
				, DAY(@date)
				, DATENAME(DAY, @date)
				, CASE WHEN DATEPART(WEEKDAY,@date) IN (1,7) THEN 0 ELSE 1 END
				, CASE WHEN DATEPART(WEEKDAY,@date) IN (1,7) THEN 'No' ELSE 'Yes' END
				)

-- Increment the date and Day ID
		SET @Date = DATEADD(DAY,1, @date)
		SET @DayID = @DayID + 1
	END
	END




/*********************************************************/
/*********************************************************/
		--  DML f.FLIGHT_TABLE (STG --> FACT)
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM f.FLIGHT_TABLE) = 0
	BEGIN
INSERT INTO f.FLIGHT_TABLE(FlightID, fkCalendar, fkAIRLINE, FKORIGIN_AIRPORT, DESTINATION_AIRPORT, Scheduled_Departure, Departure, Departure_Delay_min, Scheduled_Arrival, Arrival_time_local, ARRIVAL_DELAY_min, Flight_Length_min, Total_Time, DIVERTED, CANCELLED, CANCELLATION_Reason)

Select sFT.FlightID -- ID
	, tC.pkCalendar -- pkcalen
	, sFT.fkAIRLINE -- fkAir
	, sFT.fkORIGIN_AIRPORT -- fkorigin
	, sFT.DESTINATION_AIRPORT -- dest airport
	, sFT.Scheduled_Departure -- sched 
	, sFT.Departure -- depart
	, sFT.Departure_Delay_min -- depart delay min
	, sFT.Scheduled_Arrival --sched arri
	, sFT.Arrival_time_local -- local
	, sFT.ARRIVAL_DELAY_min -- arrival delay min
	, sFT.Flight_Length_min -- length min
	, sFT.Total_Time -- total time
	, sFT.DIVERTED -- diverted
	, sFT.CANCELLED -- cancelled
	, sFT.CANCELLATION_Reason -- can reason
FROM Flights_Reporting.stg.FLIGHT_TABLE sFT      -- Source, Staged Table 
	INNER JOIN Flight_Target.dim.Calendar tC -- Target Calendar
	ON sFT.FlightDate = tC.DateValue

	END


The statement has been terminated.

Query was canceled by user

Total execution time: 00:00:02.962